<a href="https://colab.research.google.com/github/ZaimaSartajTaheri/ResearchCodes/blob/main/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [23]:
test_dir="/content/drive/MyDrive/Project_materials/CovidDataset/Test"
train_dir="/content/drive/MyDrive/Project_materials/CovidDataset/Train"
val_dir="/content/drive/MyDrive/Project_materials/CovidDataset/Val"

In [24]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.callbacks import History 
from keras.applications import vgg16


In [25]:
from keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    # rotation_range=20,
     #width_shift_range=0.1,
    # shear_range=0.1,
     zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)

In [26]:
train = image_generator.flow_from_directory(train_dir, 
                                            batch_size=32, 
                                            shuffle=True, 
                                            class_mode='categorical',
                                            target_size=(224, 224))

validation =image_generator.flow_from_directory(val_dir, 
                                                batch_size=1, 
                                                shuffle=False, 
                                                class_mode='categorical',
                                                target_size=(224, 224))

test =image_generator.flow_from_directory(test_dir, 
                                            batch_size=1, 
                                            shuffle=False, 
                                            class_mode='categorical',
                                            target_size=(224, 224))

Found 8376 images belonging to 3 classes.
Found 2792 images belonging to 3 classes.
Found 2790 images belonging to 3 classes.


In [27]:
   
nb_train_samples = len(train.filenames)  
nb_train_samples


8376

In [28]:
num_classes=len(train.class_indices)
num_classes

3

In [29]:
 from keras.utils.np_utils import to_categorical 
 # get the class lebels for the training data, in the original order  
 train_labels = train.classes  
   
 # convert the training labels to categorical vectors  
 train_labels = to_categorical(train_labels, num_classes=num_classes) 
 train_labels 

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [30]:
 validation_labels = validation.classes  
 validation_labels = to_categorical(validation_labels, num_classes=num_classes) 
 validation_labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [31]:
# from keras.applications.vgg16 import VGG16
# from keras.preprocessing import image
# from keras.applications.vgg16 import preprocess_input
# from keras.layers import Input, Flatten, Dense
# from keras.models import Model
# import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
# model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
# model_vgg16_conv.summary()


In [32]:


# #Create your own input format (here 3x200x200)
# input = Input(shape=(224,224,3),name = 'image_input')

# #Use the generated model 
# output_vgg16_conv = model_vgg16_conv(input)

In [33]:
# from keras.layers import Dropout
# #Add the fully-connected layers 
# x = Flatten(name='flatten')(output_vgg16_conv)
# x = Dense(500, activation='relu', name='fc1')(x)
# x = Dropout(0.5)(x)
# x = Dense(100, activation='relu', name='fc2')(x)
# x = Dropout(0.3)(x)
# x = Dense(num_classes, activation='softmax', name='predictions')(x)

In [34]:
# from keras.applications.vgg16 import VGG16
# from keras.preprocessing import image
# from keras.applications.vgg16 import preprocess_input
# from keras.layers import Input, Flatten, Dense
# from keras.models import Model
# import numpy as np
# # Generate a model with all layers (with top)
# vgg16 = VGG16(weights=None, include_top=True)

# #Add a layer where input is the output of the  second last layer 
# x = Dense(num_classes, activation='softmax')(vgg16.layers[-1].output)

# #Then create the corresponding model 
# my_model = Model(vgg16.input,x)
# my_model.summary()

In [35]:
!pip install -U efficientnet

Requirement already up-to-date: efficientnet in /usr/local/lib/python3.7/dist-packages (1.1.1)


In [36]:

import efficientnet.keras as efn

In [37]:
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')


In [38]:
for layer in base_model.layers:
    layer.trainable = False

In [39]:
from keras.layers import Dropout
from keras.layers import Input, Flatten, Dense
from keras.models import Model

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation="softmax")(x)
model_final = Model(base_model.input,predictions)

In [40]:
model_final.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [41]:
model_final.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:

eff_history = model_final.fit(train, validation_data = validation, steps_per_epoch = 262, validation_steps=25 ,epochs = 10)

Epoch 1/10
262/262 [==============================] - 716s 3s/step - loss: 1.6309 - accuracy: 0.8573 - val_loss: 0.0717 - val_accuracy: 0.9600
Epoch 2/10
244/262 [==========================>...] - ETA: 49s - loss: 0.1212 - accuracy: 0.9392

In [ ]:
# #Create your own model 
# my_model = Model(input,x)

In [ ]:
# my_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 500)               12544500  
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
fc2 (Dense)                  (None, 100)               50100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0     

In [ ]:
# my_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#  history = my_model.fit(train,
#           epochs=15,   
#           validation_data=validation
#           ) 

Epoch 1/15
179/262 [===================>..........] - ETA: 10:05 - loss: 1.0997 - accuracy: 0.3384